<a href="https://colab.research.google.com/github/lisahqwang/WIMTDocMan/blob/main/DocMan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.35.2 datasets==2.15.0 scikit-learn==1.2.2
!pip install langchain-community
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully u

In [2]:
from transformers import pipeline
import torch
from torch import Tensor
from langchain_community.document_loaders import PyPDFLoader
from pypdf import PdfReader

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
def PDFloader(link):
    pdfreader = PdfReader(link)
    # read text from pdf
    raw_text = ""
    for i, page in enumerate(pdfreader.pages):
        content = page.extract_text()
        if content:
            raw_text += content

    num_of_pages = len(pdfreader.pages)
    num_of_words = len(raw_text)
    num_new_lines = raw_text.count("\n")
    period_counts = raw_text.count(".")
    print("Number of pages:", num_of_pages, "\n"
      "Number of words:", num_of_words, "\n"
      "Number of new lines:", num_new_lines, "\n"
      "Number of periods:" ,period_counts)
    #print(raw_text[0:num_of_words])
    return raw_text[0:num_of_words] #first hundred tokens so as to not overwhelm

In [4]:
def classification(sequences):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli",
                      device=device)
    #candidate_labels = ["Technology", "Business", "Legal", "Human Resources", "Tax", "Finance"]
    candidate_labels = ["Brokerage account agreements",
                        "Adding account features",
                        "Transfers distributions and payments",
                        "Retirement account forms",
                        "Tax and legal forms",
                        "Estate processing forms",
                        "Miscellaneous forms"]

    results = classifier(sequences, candidate_labels)
    return(results)

In [5]:
def Get_Top_Category(string_now):
  index_start = string_now.index('labels')
  index_end = index_start + 209
  reformed = string_now[index_start+9:index_end]
  print(reformed)
  reformed = reformed.replace('[', '')
  reformed = reformed.replace(']', '')
  reformed = reformed.replace("'", '')
  reformed = reformed.split(',')
  return(reformed[0])

In [6]:
def Output(category, path):
  start_index = path.index('src/')
  return path[start_index + 4:] + "|" + category


In [9]:
if __name__ == "__main__":
    #main function

    #Test Case 1:
    #classification(["Microsoft Pushes Off SP2 Release Microsoft will delay the release of its SP2 update for another week to fix software glitches."])

    #Test Case 2:
    #path = r'/usr/local/src/Expatriate_Certificate.pdf'

    #Test Case 3:
    path = r'/usr/local/src/Tax_and_legal_form.pdf'

    preliminary_string = classification(PDFloader(path))
    category = Get_Top_Category(str(preliminary_string))
    print(Output(category, path))



Number of pages: 2 
Number of words: 7098 
Number of new lines: 193 
Number of periods: 12


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

['Brokerage account agreements', 'Adding account features', 'Transfers distributions and payments', 'Miscellaneous forms', 'Tax and legal forms', 'Estate processing forms', 'Retirement account forms']
Tax_and_legal_form.pdf|Brokerage account agreements


In [10]:
print(preliminary_string)

{'sequence': '   \n \n \n \n \n \n   \n    \n  \n  \n     \n    \n \n   \n \n    \n \n        \n                \n \n \n  \n \n \n      \n  \n       \n        \n            \n    \n  \n    \n   \n  \n     \n  \n  \n   \n  \n     \n   \n     \n \n    \n   \n   \n   \n   \nSub Firm # Account Number \n205 \n(Office Use Only) \nWellsTrade® Corporate Resolution for Depository and Brokerage \nAccount Number \n(Client to Complete-Required) \nName of Corporation (As shown on New Account Application) Account Number \nthe undersigned, hereby \nFull Name \ncertifies: that I am , Secretary \nFull Name \nof \nFull Name of Corporation \nand validly existing under the laws of the State of \n, a corporation duly organized \n; that the following is a true copy \nof Resolutions duly adopted by the Board of Directors at a meeting duly held on \nDate \n, \nat which a quorum was present and acting throughout, or if permitted by applicable law and the Articles of Incorporation, \nCharter and/or By-Laws of t

In [14]:
print(Output(category, path))

text_file = open("Output.txt", "w")
text_file.write(Output(category, path))
text_file.close()

Tax_and_legal_form.pdf|Brokerage account agreements
